In [1]:
# pip install youtube-comment-scraper-python

In [2]:
#Data Extraction
from youtube_comment_scraper_python import *
import pandas as pd

link=input("Youtube links: ")
saved=input("Output name: ")
youtube.open(link)

response = youtube.video_comments()
all_data=[]
for i in range(0,2):   #It will scroll 2 times
    response = youtube.video_comments()
    data = response['body']
    all_data.extend(data)
df = pd.DataFrame(data)
df.to_csv(saved)


Starting DataKund...
Youtube links: https://youtu.be/jtn-hRJjl68
Output name: comments


Progress:  66%|█████████████████████████████████████████████                       | 132.6/200 [00:26<00:22,  3.03it/s]
Progress:  69%|██████████████████████████████████████████████▉                     | 138.0/200 [00:32<00:33,  1.84it/s]
Progress:   8%|█████▎                                                               | 15.4/200 [00:04<00:48,  3.82it/s]
Progress:  84%|█████████████████████████████████████████████████████████▍          | 168.8/200 [00:34<00:06,  4.65it/s]
Progress: 210.60000000000002it [00:40,  5.32it/s]                                                                      
Progress: 216.20000000000002it [00:42,  4.72it/s]                                   | 88.0/200 [00:12<00:16,  6.68it/s]
Progress: 220.60000000000002it [00:44,  4.11it/s]                                   | 93.6/200 [00:14<00:19,  5.59it/s]
Progress: 224.8it [00:46,  3.58it/s]                                                | 98.0/200 [00:16<00:22,  4.62it/s]
Progress: 229.0it [00:48,  3.16it/s]████

In [39]:
#Data Transformations
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import os

#import functions for data preprocessing and data preparation
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string
from string import punctuation
import nltk
import re

In [7]:
#Read Data
df.columns
data = pd.DataFrame(df['Comment'])
data.head()

,Comment
0,Data Science Roadmap -1. Learning a programmin...
1,I am a data Science Student and believe me the...
2,The quality of Content shared above is simply ...
3,Steps to get in data ScienceStep 1 (3:32) : Le...
4,Data Science Roadmap -1. Learning a programmin...


In [11]:
#Data labelling
nltk.download('vader_lexicon')
sentiments=SentimentIntensityAnalyzer()
data["Positive"] = [sentiments.polarity_scores(i)['pos'] for i in data['Comment']]
data["Negative"] = [sentiments.polarity_scores(i)['neg'] for i in data['Comment']]
data["Neutral"] = [sentiments.polarity_scores(i)['neu'] for i in data['Comment']]
data["Compound"] = [sentiments.polarity_scores(i)['compound'] for i in data['Comment']]
score = data["Compound"].values
sentiment = []
for i in score:
    if i>=0.05:
        sentiment.append('Positive')
    elif i<=-0.05:
        sentiment.append('Negative')
    else: 
        sentiment.append("Neutral")
data["Sentiment"] = sentiment
data.head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mamta\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Comment,Positive,Negative,Neutral,Compound,Sentiment
0,Data Science Roadmap -1. Learning a programmin...,0.000,0.0,1.000,0.0000,Neutral
1,I am a data Science Student and believe me the...,0.304,0.0,0.696,0.8625,Positive
2,The quality of Content shared above is simply ...,0.193,0.0,0.807,0.6705,Positive
3,Steps to get in data ScienceStep 1 (3:32) : Le...,0.000,0.0,1.000,0.0000,Neutral
4,Data Science Roadmap -1. Learning a programmin...,0.000,0.0,1.000,0.0000,Neutral


In [12]:
#Final data
data2 = data.drop(['Positive','Negative','Neutral','Compound'],axis=1)
data2.head()

,Comment,Sentiment
0,Data Science Roadmap -1. Learning a programmin...,Neutral
1,I am a data Science Student and believe me the...,Positive
2,The quality of Content shared above is simply ...,Positive
3,Steps to get in data ScienceStep 1 (3:32) : Le...,Neutral
4,Data Science Roadmap -1. Learning a programmin...,Neutral


In [13]:
#data transformations
stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer()
snowball_stemer = SnowballStemmer(language="english")
lzr = WordNetLemmatizer()


In [14]:
def text_preprocessing(text):
    text = text.lower()
    text = re.sub(r'\n',' ',text)
    text = re.sub('[%s]' % re.escape(punctuation),"",text)
    text = re.sub("^a-zA-Z0-9$,.","",text)
    text = re.sub(r'\s+',' ',text,flags=re.I)
    text = re.sub(r'\W',' ',text)
    text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])
    text = ' '.join([lzr.lemmatize(word) for word in word_tokenize(text)])
    return text


In [15]:
nltk.download('omw-1.4')
data_copy = data2.copy()
data_copy.Comment = data_copy.Comment.apply(lambda text: text_preprocessing(text))


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mamta\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [16]:
le = LabelEncoder()
data_copy['Sentiment'] = le.fit_transform(data_copy['Sentiment'])

In [17]:
processed_data = {'sentence':data_copy.Comment,
                  'sentiment':data_copy.Sentiment}
processed_data = pd.DataFrame(processed_data)
processed_data.head()

,sentence,sentiment
0,data science roadmap 1 learning programming la...,1
1,data science student believe thing insight im ...,2
2,quality content shared simply eyeopener cheer ...,2
3,step get data sciencestep 1 332 learn pythonst...,1
4,data science roadmap 1 learning programming la...,1


In [30]:
a = pd.DataFrame(processed_data['sentiment'].value_counts())
a

,sentiment
2,32
1,6
0,2


In [31]:
#Balancing Data
df_neutral = processed_data[(processed_data['sentiment']==1)]
df_negative = processed_data[(processed_data['sentiment']==0)]
df_positive=processed_data[(processed_data['sentiment']==2)]

df_negative_unsampled = resample(df_negative,replace=True, n_samples=max(a['sentiment']), random_state=42)
df_neutral_unsampled = resample(df_neutral,replace=True, n_samples=max(a['sentiment']), random_state=42)

final_data = pd.concat([df_negative_unsampled,df_neutral_unsampled,df_positive])


In [33]:
final_data['sentiment'].value_counts()

0    32
1    32
2    32
Name: sentiment, dtype: int64

In [34]:
corpus = []
for sentence in final_data['sentence']:
    corpus.append(sentence)
corpus[0:5]

['hi video nothing knowledge love content bigdata intern want deep dive learn please make video bigdata engineering separately',
 'informative video student new data science watching video confusion data science cleared thanx lot',
 'hi video nothing knowledge love content bigdata intern want deep dive learn please make video bigdata engineering separately',
 'hi video nothing knowledge love content bigdata intern want deep dive learn please make video bigdata engineering separately',
 'hi video nothing knowledge love content bigdata intern want deep dive learn please make video bigdata engineering separately']

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = final_data.iloc[:,-1].values


In [36]:
#Machine learning model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=0)
classifier = GaussianNB()
classifier.fit(X_train,y_train)

GaussianNB()

In [37]:
#Evaluation
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
cm

array([[11,  0,  0],
       [ 0,  8,  0],
       [ 0,  1,  9]], dtype=int64)

In [38]:
nb_score = accuracy_score(y_test,y_pred)
print('accuracy',nb_score)

accuracy 0.9655172413793104


In [40]:
pickle.dump(classifier,open('model.pkl','wb'))
model = pickle.load(open('model.pkl','rb'))

array([0, 2, 0, 1, 2, 2, 0, 2, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 2, 1, 1, 0, 2, 2])